In [11]:
import pyaudio
import numpy as np
import pandas as pd
import os
import librosa
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

audio_dataset_path = 'UrbanSound8K/audio/'
metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')


def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features
                       
extracted_features = []
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold'+str(row["fold"])+'/', str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])

extracted_features_df = pd.DataFrame(extracted_features, columns=['feature', 'class'])

X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = X_train[..., np.newaxis]  # Add channel dimension
X_test = X_test[..., np.newaxis]  # Add channel dimension


3555it [03:20, 18.77it/s]C:\Users\mural\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8323it [07:46, 24.21it/s]C:\Users\mural\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
8328it [07:46, 30.04it/s]C:\Users\mural\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [08:07, 17.90it/s]


In [12]:
# Load the pre-trained model

model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

accuracy = model.evaluate(X_test, y_test)

print("Test Accuracy:", accuracy[1])

Epoch 1/50
219/219 [==============================] - 3s 7ms/step - loss: 1.8405 - accuracy: 0.3797 - val_loss: 1.3455 - val_accuracy: 0.5821
Epoch 2/50
219/219 [==============================] - 1s 6ms/step - loss: 1.3106 - accuracy: 0.5513 - val_loss: 0.9841 - val_accuracy: 0.6852
Epoch 3/50
219/219 [==============================] - 1s 6ms/step - loss: 1.1156 - accuracy: 0.6329 - val_loss: 0.7790 - val_accuracy: 0.7596
Epoch 4/50
219/219 [==============================] - 1s 6ms/step - loss: 0.9369 - accuracy: 0.6923 - val_loss: 0.6913 - val_accuracy: 0.7796
Epoch 5/50
219/219 [==============================] - 2s 7ms/step - loss: 0.8577 - accuracy: 0.7218 - val_loss: 0.6460 - val_accuracy: 0.7916
Epoch 6/50
219/219 [==============================] - 2s 7ms/step - loss: 0.7524 - accuracy: 0.7522 - val_loss: 0.5896 - val_accuracy: 0.8100
Epoch 7/50
219/219 [==============================] - 1s 6ms/step - loss: 0.7052 - accuracy: 0.7689 - val_loss: 0.5093 - val_accuracy: 0.8334
Epoch 

In [3]:
filename="UrbanSound8K/14113-4-0-0.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=np.argmax(model.predict(mfccs_scaled_features), axis=-1)
#predicted_label=model.predict_classes(mfccs_scaled_features)
print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
print(prediction_class[0])

[-3.5755768e+02  5.2384933e+01 -4.1709915e+01  4.3974846e+01
 -1.6069826e+01  9.7353554e+00 -5.0241313e+00  1.0326746e+01
 -1.6136753e+01  2.6916557e+01 -1.4883501e+01  1.9002371e+01
 -1.3996512e+01  6.6752768e+00 -1.7345709e+01  7.7119689e+00
 -1.6728811e+01  1.0445915e+01 -1.6504908e+01  1.0037301e+01
 -6.7588978e+00  3.2604740e+00 -6.0810432e+00  4.5140672e+00
 -4.7407560e+00 -5.8550125e-01 -3.1900070e+00  4.0783095e+00
 -3.4920244e+00  3.5106187e+00 -5.0360985e+00  1.5612836e+00
 -4.6750493e+00  3.6949618e+00 -3.9920721e+00  4.5324945e+00
 -3.9963019e+00  2.9155529e+00 -3.2510986e+00 -2.3989076e-02]
[[-3.5755768e+02  5.2384933e+01 -4.1709915e+01  4.3974846e+01
  -1.6069826e+01  9.7353554e+00 -5.0241313e+00  1.0326746e+01
  -1.6136753e+01  2.6916557e+01 -1.4883501e+01  1.9002371e+01
  -1.3996512e+01  6.6752768e+00 -1.7345709e+01  7.7119689e+00
  -1.6728811e+01  1.0445915e+01 -1.6504908e+01  1.0037301e+01
  -6.7588978e+00  3.2604740e+00 -6.0810432e+00  4.5140672e+00
  -4.7407560e+00 

In [13]:
import pyaudio
import numpy as np
import librosa
import time

# Function to extract features from real-time audio
def real_time_feature_extractor(audio):
    sample_rate = 22050  # Adjust this based on the sample rate of your microphone
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    return mfccs_scaled_features


def predict_real_time(audio_chunk): # Function to perform real-time prediction
  # Extract features from the audio chunk
  features = real_time_feature_extractor(audio_chunk)
  features = features[np.newaxis, ..., np.newaxis] # Add channel dimension for compatibility with the model
  prediction = model.predict(features) # Make a prediction
  predicted_class_index = np.argmax(prediction)  # Get the predicted class and confidence level
  confidence = np.max(prediction)
  predicted_class_label = labelencoder.classes_[predicted_class_index] # Convert predicted class index back to the original class label
  return predicted_class_label, confidence

# Set up PyAudio
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paFloat32,
                channels=1,
                rate=22050,  # Adjust this based on the sample rate of your microphone
                input=True,
                frames_per_buffer=1024)

def Function_A(_):
    print("Audio Classification is running...")
    continuous_count = 0
    previous_prediction = None
    while True:
      try:
        audio_chunk = np.frombuffer(stream.read(2048), dtype=np.float32)
        predicted_class, confidence = predict_real_time(audio_chunk)

        if confidence >= 0.999:
          if predicted_class == previous_prediction:
            continuous_count += 1
            if continuous_count >= 5:
              print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2%}")
              continuous_count = 0  # Reset count after printing
          else:
            continuous_count = 0  # Reset count if prediction changes
          previous_prediction = predicted_class
          print("Sound not recognized")
        else:
          print("Sound not recognized")
          continuous_count = 0  # Reset count if confidence is below threshold

      except KeyboardInterrupt:
        print("Stopping real-time predictions...")
        break

In [4]:
import speech_recognition as sr

def Speech_recording(timeout=5):
    # Initialize the recognizer
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Please start speaking...")
        try:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source, timeout=timeout)
            text = recognizer.recognize_google(audio) # Google Web Speech API for recognition
            print("Text:", text)
            return text

        except sr.WaitTimeoutError:
            print("Speech recognition timed out. Program will now exit.")
            exit(1)

        except sr.UnknownValueError:
            print("Speech not recognized. Please try again.")
            return None

#if __name__ == "__main__":
#    recognition_timeout = 5 # in seconds
#    recognized_text = recognize_speech(timeout=recognition_timeout)
Speech_recording(timeout=5)

Please start speaking...
Speech not recognized. Please try again.


In [14]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd

### LOADING THE VOICE DATA FOR VISUALIZATION ###
#walley_sample = "audio_data/audio_data_1.wav"
#data, sample_rate = librosa.load(walley_sample)

##### VISUALIZING WAVE FORM ##
#plt.title("Wave Form")
#librosa.display.waveplot(data, sr=sample_rate)
#plt.show()

##### VISUALIZING MFCC #######
#mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
#print("Shape of mfcc:", mfccs.shape)

#plt.title("MFCC")
#librosa.display.specshow(mfccs, sr=sample_rate, x_axis='time')
#plt.show()

##### Doing this for every sample ##

all_data = []

data_path_dict = {
    0: ["background_sound/" + file_path for file_path in os.listdir("background_sound/")],
    1: ["audio_data/" + file_path for file_path in os.listdir("audio_data/")]
}

# the background_sound/ directory has all sounds which DOES NOT CONTAIN wake word
# the audio_data/ directory has all sound WHICH HAS Wake word

for class_label, list_of_files in data_path_dict.items():
    for single_file in list_of_files:
        audio, sample_rate = librosa.load(single_file) ## Loading file
        mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40) ## Apllying mfcc
        mfcc_processed = np.mean(mfcc.T, axis=0) ## some pre-processing
        all_data.append([mfcc_processed, class_label])
    print(f"Info: Succesfully Preprocessed Class Label {class_label}")

df = pd.DataFrame(all_data, columns=["feature", "class_label"])

###### SAVING FOR FUTURE USE ###
df.to_pickle("final_audio_data_csv/audio_data.csv")

Info: Succesfully Preprocessed Class Label 0
Info: Succesfully Preprocessed Class Label 1


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn.metrics import confusion_matrix, classification_report
#from plot_cm import plot_confusion_matrix

##### Loading saved csv ##############
df = pd.read_pickle("final_audio_data_csv/audio_data.csv")

####### Making our data training-ready
X = df["feature"].values
X = np.concatenate(X, axis=0).reshape(len(X), 40)

y = np.array(df["class_label"].tolist())
y = to_categorical(y)

####### train test split ############
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Training ############

model_WWD = Sequential([
    Dense(256, input_shape=X_train[0].shape),
    Activation('relu'),
    Dropout(0.5),
    Dense(256),
    Activation('relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

print(model_WWD.summary())

model_WWD.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

print("Model Score: \n")
history = model_WWD.fit(X_train, y_train, epochs=1000)
score = model_WWD.evaluate(X_test, y_test)
print(score)

#### Evaluating our model ###########
#print("Model Classification Report: \n")
#y_pred = np.argmax(model.predict(X_test), axis=1)
#cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
#print(classification_report(np.argmax(y_test, axis=1), y_pred))
#plot_confusion_matrix(cm, classes=["Does not have Wake Word", "Has Wake Word"])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 256)               10496     
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 256)               65792     
                                                                 
 activation_5 (Activation)   (None, 256)               0         
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 2)                

8/8 [==============================] - 0s 5ms/step - loss: 0.3273 - accuracy: 0.8770
Epoch 70/1000
8/8 [==============================] - 0s 5ms/step - loss: 0.2971 - accuracy: 0.8651
Epoch 71/1000
8/8 [==============================] - 0s 5ms/step - loss: 0.2940 - accuracy: 0.8730
Epoch 72/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.4042 - accuracy: 0.8810
Epoch 73/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.2489 - accuracy: 0.9127
Epoch 74/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.3107 - accuracy: 0.8810
Epoch 75/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.2620 - accuracy: 0.8929
Epoch 76/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.2307 - accuracy: 0.9127
Epoch 77/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.3311 - accuracy: 0.8690
Epoch 78/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.3121 - accuracy: 0.9008
Epoch 79/1000
8/8 [=====

8/8 [==============================] - 0s 3ms/step - loss: 0.1329 - accuracy: 0.9405
Epoch 152/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1227 - accuracy: 0.9484
Epoch 153/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1514 - accuracy: 0.9484
Epoch 154/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1518 - accuracy: 0.9405
Epoch 155/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1180 - accuracy: 0.9563
Epoch 156/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1072 - accuracy: 0.9563
Epoch 157/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1328 - accuracy: 0.9405
Epoch 158/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0976 - accuracy: 0.9524
Epoch 159/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1458 - accuracy: 0.9365
Epoch 160/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.1603 - accuracy: 0.9444
Epoch 161/1000


8/8 [==============================] - 0s 3ms/step - loss: 0.1213 - accuracy: 0.9643
Epoch 234/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0810 - accuracy: 0.9802
Epoch 235/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0657 - accuracy: 0.9643
Epoch 236/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0716 - accuracy: 0.9722
Epoch 237/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0847 - accuracy: 0.9603
Epoch 238/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0703 - accuracy: 0.9762
Epoch 239/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0569 - accuracy: 0.9802
Epoch 240/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0593 - accuracy: 0.9762
Epoch 241/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0508 - accuracy: 0.9762
Epoch 242/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0705 - accuracy: 0.9841
Epoch 243/1000


8/8 [==============================] - 0s 3ms/step - loss: 0.0224 - accuracy: 0.9921
Epoch 316/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0145 - accuracy: 0.9921
Epoch 317/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0197 - accuracy: 0.9960
Epoch 318/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0334 - accuracy: 0.9841
Epoch 319/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0774 - accuracy: 0.9722
Epoch 320/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0225 - accuracy: 0.9921
Epoch 321/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0257 - accuracy: 0.9841
Epoch 322/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0204 - accuracy: 0.9921
Epoch 323/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0109 - accuracy: 0.9960
Epoch 324/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0318 - accuracy: 0.9802
Epoch 325/1000


...

8/8 [==============================] - 0s 4ms/step - loss: 0.0391 - accuracy: 0.9841
Epoch 644/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0391 - accuracy: 0.9960
Epoch 645/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0143 - accuracy: 0.9960
Epoch 646/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 647/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0313 - accuracy: 0.9921
Epoch 648/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0168 - accuracy: 0.9921
Epoch 649/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0243 - accuracy: 0.9921
Epoch 650/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0365 - accuracy: 0.9921
Epoch 651/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0204 - accuracy: 0.9881
Epoch 652/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0110 - accuracy: 0.9960
Epoch 653/1000


8/8 [==============================] - 0s 3ms/step - loss: 0.0394 - accuracy: 0.9802
Epoch 726/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 727/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0112 - accuracy: 0.9960
Epoch 728/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0096 - accuracy: 0.9960
Epoch 729/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0153 - accuracy: 0.9960
Epoch 730/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0140 - accuracy: 0.9960
Epoch 731/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0148 - accuracy: 0.9960
Epoch 732/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0235 - accuracy: 0.9881
Epoch 733/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 0.9921
Epoch 734/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0169 - accuracy: 0.9960
Epoch 735/1000


8/8 [==============================] - 0s 3ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 808/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 809/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0139 - accuracy: 0.9921
Epoch 810/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 811/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0146 - accuracy: 0.9960
Epoch 812/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0150 - accuracy: 0.9960
Epoch 813/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9960
Epoch 814/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0224 - accuracy: 0.9960
Epoch 815/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0392 - accuracy: 0.9921
Epoch 816/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 817/1000


8/8 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 890/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 891/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0119 - accuracy: 0.9960
Epoch 892/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0235 - accuracy: 0.9921
Epoch 893/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 894/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0189 - accuracy: 0.9921
Epoch 895/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 896/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0125 - accuracy: 0.9960
Epoch 897/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 898/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0110 - accuracy: 0.9960
Epoch 899/1000


Epoch 971/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 972/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 973/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 974/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 975/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0239 - accuracy: 0.9881
Epoch 976/1000
8/8 [==============================] - 0s 3ms/step - loss: 0.0126 - accuracy: 0.9960
Epoch 977/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0374 - accuracy: 0.9881
Epoch 978/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 979/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 980/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0233 - accuracy: 0.9960


In [17]:
import os
import librosa
import pyaudio
from tensorflow.keras.models import load_model

# Set up PyAudio
chunk_size = 1024
sample_format = pyaudio.paFloat32
channels = 1
fs = 44100
p = pyaudio.PyAudio()


# Open stream
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk_size,
                input=True)

#print("Listening for wake word...")

def Wake_Word_Detection(_):
    while True:
        try:
            # Read audio data from the microphone
            data = stream.read(chunk_size)
            audio_input = np.frombuffer(data, dtype=np.float32)

            # Preprocess the audio input
            mfcc = librosa.feature.mfcc(y=audio_input, sr=fs, n_mfcc=40)
            mfcc_processed = np.mean(mfcc.T, axis=0)
            input_features = mfcc_processed.reshape(1, -1)

            # Make prediction
            prediction = model_WWD.predict(input_features)

            # Get the class label with the highest confidence
            prediction_class = np.argmax(prediction)
            confidence = np.max(prediction) * 100

            # Print result
            if confidence > 99.9:
                if prediction_class == 1:
                    print("Wake word detected with confidence:", confidence)
                else:
                    print("Wake word NOT detected")
            else:
                print("Wake word NOT detected")

        except KeyboardInterrupt:
            print("Stopping the wake word detection.")
            break

    # Stop stream and close PyAudio
stream.stop_stream()
stream.close()
p.terminate()

In [18]:
import os
import librosa
import pyaudio
import numpy as np
from tensorflow.keras.models import load_model
import speech_recognition as sr

# Set up PyAudio
chunk_size = 1024
sample_format = pyaudio.paFloat32
channels = 1
fs = 44100
p = pyaudio.PyAudio()

# Load wake word detection model
#model_WWD = load_model("your_model_path.h5")  # Replace with the actual path to your wake word detection model

# Open stream
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk_size,
                input=True)

def wake_word_detection():
    while True:
        try:
            # Read audio data from the microphone
            data = stream.read(chunk_size)
            audio_input = np.frombuffer(data, dtype=np.float32)

            # Preprocess the audio input
            mfcc = librosa.feature.mfcc(y=audio_input, sr=fs, n_mfcc=40)
            mfcc_processed = np.mean(mfcc.T, axis=0)
            input_features = mfcc_processed.reshape(1, -1)

            # Make prediction
            prediction = model_WWD.predict(input_features)

            # Get the class label with the highest confidence
            prediction_class = np.argmax(prediction)
            confidence = np.max(prediction) * 100

            # Return result
            if confidence == 100 and prediction_class == 1:
                return True  # Wake word detected

        except KeyboardInterrupt:
            print("Stopping the wake word detection.")
            break

    return False

def speech_recognition():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Please start speaking...")
        try:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)  # Google Web Speech API for recognition
            print("Text:", text)
            return text

        except sr.WaitTimeoutError:
            print("Speech recognition timed out. No speech detected.")
            return None

        except sr.UnknownValueError:
            print("Speech not recognized. Please try again.")
            return None

def Function_B():
    while True:
        # Wake word detection
        if wake_word_detection():
            print("Wake word detected. Starting speech recognition...")
            recognized_text = speech_recognition()

            # If speech is detected, stop wake word detection temporarily
            if recognized_text is not None:
                print("Speech detected. Pausing wake word detection.")
            else:
                print("No speech detected. Resuming wake word detection.")

In [ ]:
import threading
import tkinter as tk


def run_functions():
    output_a.set(Function_A(_))
    output_b.set(Function_B())

# Create the Tkinter GUI window
root = tk.Tk()
root.title("Parallel Function Execution")

# Tkinter variables to hold the outputs of A() and B()
output_a = tk.StringVar()
output_b = tk.StringVar()

# Create labels to display the outputs
label_a = tk.Label(root, textvariable=output_a, justify="left")
label_b = tk.Label(root, textvariable=output_b, justify="right")

# Create a button to run the functions in parallel
run_button = tk.Button(root, text="Run Functions", command=run_functions)

# Pack the widgets into the GUI window
label_a.pack(side="left", padx=10)
label_b.pack(side="right", padx=10)
run_button.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()


Audio Classification is running...
1/1 [==============================] - 0s 156ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step
Sound not recognized
1/1 [==============================] - 0s 29ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step
Sound not recognized
1/1 [==============================] - 0s 24ms/step
Sound not recognized
1/1 [==============================] - 0s 21ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 29ms/step
Sound not recognized
1/1 [==============================] - 0s 38ms/step
Sound not recognized
1/1 [==============================] - 0s 39ms/step
Sound not recognized
1/1 [==============================] - 0s 30ms/step
Sound not recognized
1/1 [==========

1/1 [==============================] - 0s 26ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 23ms/step
Sound not recognized
1/1 [==============================] - 0s 25ms/step
Sound not recognized
1/1 [==============================] - 0s 29ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step
Sound not recognized
1/1 [==============================] - 0s 28ms/step
Sound not recognized
1/1 [==============================] - 0s 29ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step

1/1 [==============================] - 0s 41ms/step
Sound not recognized
1/1 [==============================] - 0s 31ms/step
Sound not recognized
1/1 [==============================] - 0s 33ms/step
Sound not recognized
1/1 [==============================] - 0s 35ms/step
Sound not recognized
1/1 [==============================] - 0s 25ms/step
Sound not recognized
1/1 [==============================] - 0s 37ms/step
Sound not recognized
1/1 [==============================] - 0s 31ms/step
Sound not recognized
1/1 [==============================] - 0s 24ms/step
Sound not recognized
1/1 [==============================] - 0s 32ms/step
Sound not recognized
1/1 [==============================] - 0s 30ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step
Sound not recognized
1/1 [==============================] - 0s 29ms/step
Sound not recognized
1/1 [==============================] - 0s 27ms/step
Sound not recognized
1/1 [==============================] - 0s 26ms/step

C:\Users\mural\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1024
  warnings.warn(


1/1 [==============================] - 0s 28ms/step
Wake word detected. Starting speech recognition...
Please start speaking...
Text: Malayalam lyrics
Speech detected. Pausing wake word detection.
1/1 [==============================] - 0s 28ms/step
Stopping the wake word detection.
1/1 [==============================] - 0s 23ms/step


...

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 25ms/step
Wake word detected. Starting speech recognition...
Please start speaking...


In [ ]:
import threading
import tkinter as tk

# Function A implementation
def Function_A(_):
        print("Audio Classification is running...")
    continuous_count = 0
    previous_prediction = None
    while True:
      try:
        audio_chunk = np.frombuffer(stream.read(2048), dtype=np.float32)
        predicted_class, confidence = predict_real_time(audio_chunk)

        if confidence >= 0.999:
          if predicted_class == previous_prediction:
            continuous_count += 1
            if continuous_count >= 5:
              print(f"Predicted Class: {predicted_class}, Confidence: {confidence:.2%}")
              continuous_count = 0  # Reset count after printing
          else:
            continuous_count = 0  # Reset count if prediction changes
          previous_prediction = predicted_class
          print("Sound not recognized")
        else:
          print("Sound not recognized")
          continuous_count = 0  # Reset count if confidence is below threshold

      except KeyboardInterrupt:
        print("Stopping real-time predictions...")
        break
    pass

# Function B implementation
def Function_B():
    while True:
        # Wake word detection
        if wake_word_detection():
            print("Wake word detected. Starting speech recognition...")
            recognized_text = speech_recognition()

            # If speech is detected, stop wake word detection temporarily
            if recognized_text is not None:
                print("Speech detected. Pausing wake word detection.")
            else:
                print("No speech detected. Resuming wake word detection.")
    pass

# Function to run both functions in parallel
def run_functions():
    # Create threads for each function
    thread_a = threading.Thread(target=Function_A, args=(_,))
    thread_b = threading.Thread(target=Function_B)

    # Start the threads
    thread_a.start()
    thread_b.start()

# Create the Tkinter GUI window
root = tk.Tk()
root.title("Parallel Function Execution")

# Tkinter variables to hold the outputs of A() and B()
output_a = tk.StringVar()
output_b = tk.StringVar()

# Create labels to display the outputs
label_a = tk.Label(root, textvariable=output_a, justify="left")
label_b = tk.Label(root, textvariable=output_b, justify="right")

# Create a button to run the functions in parallel
run_button = tk.Button(root, text="Run Functions", command=run_functions)

# Pack the widgets into the GUI window
label_a.pack(side="left", padx=10)
label_b.pack(side="right", padx=10)
run_button.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()
